# Machine Actionable Data Management Plan connections

Data management plans (DMPs) are documents accompanying research proposals and project outputs. DMPs are created as free-form text and describe the data and tools employed in scientific investigations. They are often seen as an administrative exercise and not as an integral part of research practice. Machine Actionable DMPs (maDMPs) takes the DMP concept further by using PIDs and PIDs services to connected al resources associated with the DMPs.

This notebook displays in a human-friendly way the connections embedded in a maDMP. By the end of this notebook, you will be able to succinctly display the essential components of the maDMP vision using persistent identifiers (PIDs): Open Researcher and Contributor IDs (ORCIDs), funders IDs, organizations Org IDs, Dataset IDs (DOIs). The demostrate this we use a dummy example from a ficticious DMP, viz. https://doi.org/10.4124/test/.879w8. The notebook fetch all the PIDs associated with this DMP displays it in a Tree Diagram. See below. The diagram puts the DMP at center and there are four main branches: datasets, funders, organisations, and people. Each branch gives birth to indiviual entities of those branches. For example, the name of all the people that contributed to the DMP.  


<img src="example_plot.png"/>


The process of displaying the DMP visulisation is very simple. First, and after a initial setup, we fetch all the we need from the DataCite GraphQL API. Then, we transform this data into a data structure that can be use for visulisation. Finally, we take the data tranformation and supply it to a Vega visulisation specification to generate the Chart you can see above. 




In [25]:
%%capture
# Install required Python packages
!pip install dfply altair altair_saver vega altair_viewer

In [112]:
import json
import pandas as pd
import numpy as np
from dfply import *
import altair.vega.v5 as alt
from altair_saver import save
# alt.renderers.enable('notebook')



In [127]:
# Prepare the GraphQL client
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

## Fetching Data

We obtain all the data from the DataCite GraphQL API.


In [198]:
# Generate the GraphQL query to retrieve up to 100 outputs of University of Oxford, with at least 100 views each.
query_params = {
    "id" : "https://doi.org/10.48321/D1H59R",
    "maxOutputs": 100,
    "minViews" : 100
}

query = gql("""query getOutputs($id: ID!)
{
 dataManagementPlan(id: $id) {
    id
    name: titles(first:1) {
      title
    }
    datasets: references(resourceTypeId:"dataset") {
        totalCount
        nodes {
          id: doi
          name: titles(first:1)  {
            title
          }
        }
      }
    publications: relatedIdentifiers{
      id: relatedIdentifier
      name: relatedIdentifier
      relationType
    }
    hostingInstitution: contributors(contributorType: "HostingInstitution") {
      id
      name
    }
    producer: contributors(contributorType: "Producer") {
      id
      name
    }   
    fundingReferences {
      id: funderIdentifier
      name: funderName
    }
    creators {
      id
      name
      affiliation{
        id
      }
    } 
    contributors {
      id
      name
      affiliation{
        id
      }
    }     
  }
}
""")

In [199]:
def get_data():
    return client.execute(query, variable_values=json.dumps(query_params))["dataManagementPlan"]


## Data Transformation

Simple transformations are performed to convert the graphql response into an array that can be take by Vega.

In [211]:
def add_node_attributes(dataframe, parent=2):
    """Modifies each item to include attributes needed for the node visulisation

    Parameters:
    dataframe (dataframe): A dataframe with all the itemss
    parent (int): The id of the parent node

    Returns:
    dataframe:Returning vthe same dataframe with new attributes

   """
    if (dataframe) is None:
        return pd.DataFrame() 
    else: 
        print(dataframe) 
        return (dataframe >>
        mutate(
        id = X.id,
        tooltip = X.id,
        parent = parent,
        ))
  

In [212]:
def create_node(array=[], parent=2):
    """creates a node for the chart and formats it

    Parameters:
    array (array): An array with all the itemss
    parent (int): The id of the parent node

    Returns:
    dict:Dict with all the nodes

   """
    print(array)
    if len(array) == 0:
        return {} 
    else:
        # return {} if (array) is None else array
        df = add_node_attributes(pd.DataFrame(array,columns=array[0].keys()), parent)
        return df.to_dict(orient='records')
     

In [213]:
def merge_nodes(dmpTitle,id,dataset=[],references=[],funders=[],orgs=[],people=[]):
    """Merges all the nodes lists

    Parameters:
    datasets (array): dataset nodes
    funders (array): funders nodes
    orgs (array): orgs nodes
    people (array): people nodes

    Returns:
    array:Array with all the nodes

   """
    dataset = [] if len(dataset) == 0 else dataset
    references = [] if len(references) == 0 else references
    funders = [] if len(funders) == 0 else funders
    orgs = [] if len(orgs) == 0 else orgs
    people = [] if len(people) == 0 else people

    dmp = {"id":id, "name": dmpTitle}
    datasets_node = {"id":2, "name": "Datasets", "parent":id}
    references_node = {"id":6, "name": "References", "parent":id}
    funders_node = {"id":3, "name": "Funders", "parent":id}
    organisations_node = {"id":4, "name": "Organisations", "parent":id}
    people_node = {"id":5, "name": "People", "parent":id}
    nodes_list = [dmp, datasets_node, references_node, funders_node,organisations_node,people_node] + dataset + references + funders + orgs + people,
    # return np.array(nodes_list, dtype=object)    
    return nodes_list[0]

In [214]:
data = get_data()
print(data)
datasets = create_node(data["datasets"]["nodes"],2)
references = create_node(data["publications"],6)
orgs = create_node(data["hostingInstitution"],4)
people = create_node((data["creators"] + data["contributors"]) ,5)
dmp_title = " " ##data["name"][0]["title"]
funders = create_node(data["fundingReferences"],3)
id = data["id"]
nodes = merge_nodes(dmp_title,id, datasets, references, funders, orgs, people)

{'id': 'https://doi.org/10.48321/d1h59r', 'name': [{'title': 'LTREB: Drivers of temperate forest carbon storage from canopy closure through successional time'}], 'datasets': {'totalCount': 0, 'nodes': []}, 'publications': [{'id': '10.1893/0005-3155-88.3.10', 'name': '10.1893/0005-3155-88.3.10', 'relationType': 'IsPartOf'}, {'id': '10.1111/nph.15227', 'name': '10.1111/nph.15227', 'relationType': 'IsPartOf'}, {'id': '10.1893/0005-3155-88.3.10', 'name': '10.1893/0005-3155-88.3.10', 'relationType': 'IsPartOf'}, {'id': '10.1139/cjfr-2018-0211', 'name': '10.1139/cjfr-2018-0211', 'relationType': 'IsPartOf'}, {'id': '10.1007/s00442-018-4236-x', 'name': '10.1007/s00442-018-4236-x', 'relationType': 'IsPartOf'}, {'id': '10.6073/pasta/e43651bde3564e402cd0595e3ef26994', 'name': '10.6073/pasta/e43651bde3564e402cd0595e3ef26994', 'relationType': 'IsPartOf'}], 'hostingInstitution': [{'id': 'https://ror.org/03yrm5c26', 'name': 'California Digital Library'}], 'producer': [{'id': 'https://ror.org/00jmfr29

## Visulization

All transofrmed data is then feeded into a Vega specification for display.





In [224]:
def vega_template(data):
    """Injects data into the vega specification

    Parameters:
    data (array): Array of nodes

    Returns:
    VegaSpec:Specification with data

   """
    return """
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "An example of a radial layout for a node-link diagram of hierarchical data.",
  "width": 920,
  "height": 720,
  "padding": 5,
  "autosize": "none",
  "signals": [
    {"name": "labels", "value": true, "bind": {"input": "checkbox"}},
    {
      "name": "radius",
      "value": 280,
      "bind": {"input": "range", "min": 20, "max": 600}
    },
    {
      "name": "extent",
      "value": 360,
      "bind": {"input": "range", "min": 0, "max": 360, "step": 1}
    },
    {
      "name": "rotate",
      "value": 0,
      "bind": {"input": "range", "min": 0, "max": 360, "step": 1}
    },
    {
      "name": "layout",
      "value": "cluster",
      "bind": {"input": "radio", "options": ["tidy", "cluster"]}
    },
    {
      "name": "links",
      "value": "orthogonal",
      "bind": {
        "input": "select",
        "options": ["line", "curve", "diagonal", "orthogonal"]
      }
    },
    {"name": "originX", "update": "width / 2"},
    {"name": "originY", "update": "height / 2"}
  ],
  "data": [
    {
      "name": "tree",
      "values": """ + data + """,
      "transform": [
        {"type": "stratify", "key": "id", "parentKey": "parent"},
        {
          "type": "tree",
          "method": {"signal": "layout"},
          "size": [1, {"signal": "radius"}],
          "as": ["alpha", "radius", "depth", "children"]
        },
        {
          "type": "formula",
          "expr": "(rotate + extent * datum.alpha + 270) % 360",
          "as": "angle"
        },
        {"type": "formula", "expr": "PI * datum.angle / 180", "as": "radians"},
        {
          "type": "formula",
          "expr": "inrange(datum.angle, [90, 270])",
          "as": "leftside"
        },
        {
          "type": "formula",
          "expr": "originX + datum.radius * cos(datum.radians)",
          "as": "x"
        },
        {
          "type": "formula",
          "expr": "originY + datum.radius * sin(datum.radians)",
          "as": "y"
        }
      ]
    },
    {
      "name": "links",
      "source": "tree",
      "transform": [
        {"type": "treelinks"},
        {
          "type": "linkpath",
          "shape": {"signal": "links"},
          "orient": "radial",
          "sourceX": "source.radians",
          "sourceY": "source.radius",
          "targetX": "target.radians",
          "targetY": "target.radius"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "color",
      "type": "linear",
      "range": {"scheme": "viridis"},
      "domain": {"data": "tree", "field": "depth"},
      "zero": true
    }
  ],
  "marks": [
    {
      "type": "path",
      "from": {"data": "links"},
      "encode": {
        "update": {
          "x": {"signal": "originX"},
          "y": {"signal": "originY"},
          "path": {"field": "path"},
          "stroke": {"value": "#ccc"}
        }
      }
    },
    {
      "type": "symbol",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "size": {"value": 300}, "stroke": {"value": "#fff"}
          },
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "fill": {"scale": "color", "field": "depth"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "name"},
          "fontSize": {"value": 12},
          "baseline": {"value": "middle"},
          "tooltip": {"signal": 
          "{'Identifier': datum.tooltip}"}
        },
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "dx": {"signal": "(datum.leftside ? -1 : 1) * 12"},
          "align": {"signal": "datum.leftside ? 'right' : 'left'"},
          "opacity": {"signal": "labels ? 1 : 0"}
        }
      }
    }
  ]
}

"""

In [225]:
print(dmp_title)

In [226]:
chart = alt.vega(json.loads(vega_template(json.dumps(nodes))))

<Vega 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


A series of sliders and option are included to interact with the visulisation is displayed. One ca nremove the labels, rotate the nodes, zoom in/out, and adjust the layout.